## Dependencies required are pyaudio, speechrecognition, playsound, gTTS

In [32]:
#vital imports and one time access files

import os, pyaudio, wave, time, csv, pandas as pd, speech_recognition as sr
from gtts import gTTS
from playsound import playsound
from threading import Thread
recognizer=sr.Recognizer()
    

# probable_ans=pd.read_csv("probable_answers.csv")
# temp=probable_ans.iloc[:,1:]
# ans=temp.to_numpy().tolist()

probable_ans2=pd.read_csv("probable_ans2.csv", header=None)
ans2=probable_ans2.to_numpy().tolist()

no_of_questions=(len([name for name in os.listdir('questions/')]))

In [33]:
def recognize(filename, lang="ne-NP"):
    my_file=sr.AudioFile(filename)
    with my_file as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.record(source)
    try:
        output=recognizer.recognize_google(audio, language=lang, show_all=True)
    except sr.RequestError:
        output="API unavailable or did not response"
    except sr.UnknownValueError:
        output="Did not recognize the audio"
    return output

In [34]:
def record_and_save(num):
    time.sleep(1)
    print(num)
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 2
    fs = 44100
    seconds = 5
    filename = folder+"/"+str(num)+".wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []


    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)


    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

In [47]:
#record here
from random import randint
phone_num=randint(9800000000,9869999999)
folder="recorded/"+str(phone_num)
os.mkdir(folder)

for i in range(0, no_of_questions):
    temp="questions/"+str(i)+".mp3"
    
    #thread creation
    t1=Thread(target=playsound, args=(temp,))
    t2=Thread(target=record_and_save, args=(i,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [53]:
#recognize here
output=[]
for i in range(0, no_of_questions):
    temp=[]
    filename="recorded/"+str(phone_num)+"/"+str(i)+".wav"
    converted=recognize(filename)
    if(len(converted)==0):
        output.append("err")
    else:
        for a in converted['alternative']:
            temp.append(a['transcript'])
        output.append(temp)

        
        
#append to csv here
import numpy as np
results=[phone_num]
results_bool=[phone_num]

for i in range(0, no_of_questions):
#     c = [value for value in ans[i] if value in output[i]]
    c = [value for value in ans2[0] if value in output[i]]
    if output[i]=="err":
        results.append("err")
        results_bool.append("err")
    else:
        if len(c)>0:
            results.append(c[0])
            #check if c[0] is 1 or 0
            ind=ans2[0].index(c[0])
            results_bool.append(ans2[1][ind])
        else:
            results.append("*")
            results_bool.append("*")

with open(r'results.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(results)
    

with open(r'results_bool.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(results_bool)

print(output)
print()
print(results)
print()
print(results_bool)

[['परेको छ', 'परेकोछ', 'परीको छ', 'pariko छ', 'परिको छ'], ['छैन', 'हैन', 'होइन', 'भइन', 'छैनन्'], ['लागेको छ', 'लागेकोछ', 'बगेको छ', 'मागेको छ', 'आदिको छ'], ['छैन', 'chhaina'], 'err', ['छैन', 'हैन', 'बैठक', 'chaina', 'chhaina'], ['chhup', 'chhop', 'छप', 'थप', 'छ'], ['ह', 'home', 'हो', 'ho', 'who'], ['छैन', 'chhaina'], ['भएको छैन', 'परेको छैन', 'पढेको छैन', 'भएको छैन्ः'], 'err', ['छ', '6', '16', 'छन्', '66'], ['छैन', 'chhaina'], 'err', ['सप्तरीमा विदेश भ्रमण गर्नुभएको छ', 'सप्तरीमा विदेश भ्रमण गर्नु भएको छ', 'धादिङमा विदेश भ्रमण गर्नु भएको छ', 'धादिङमा विदेश भ्रमण गर्नुभएको छ', 'दिनमा विदेश भ्रमण गर्नु भएको छ']]

[9834765726, '*', 'छैन', 'बगेको छ', 'छैन', 'err', 'छैन', 'छ', '*', 'छैन', '*', 'err', 'छ', 'छैन', 'err', '*']

[9834765726, '*', '0', '1', '0', 'err', '0', '1', '*', '0', '*', 'err', '1', '0', 'err', '*']


In [57]:
import pandas as pd
pd.read_csv("results.csv", header=None)

,phone num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
1,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
2,9869231782,छ,छैन,*,*,*,err,*,छ,*,*,छैन,छैन,छैन,*,छैन
3,9818991368,err,छैन,*,err,*,छैन,*,*,छ,*,*,err,*,err,err
4,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
5,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*
6,9834765726,*,छैन,बगेको छ,छैन,err,छैन,छ,*,छैन,*,err,छ,छैन,err,*


In [56]:
pd.read_csv("results_bool.csv", header=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,9834765726,*,0,1,0,err,0,1,*,0,*,err,1,0,err,*


In [ ]:
# questions = ["के तपाईको टाउको दुख्छ", "के तपाइलाई ज्वरो छ ", "के तपाई को नाक बग्छ ", "के तपाइलाई सास फेर्न समस्या छ "]
# for i in range(0, len(questions)):
#     speech = gTTS(text[i], lang = "en-IN", slow = False)
#     name="questions/"+str(i)+".mp3"
#     speech.save(name)

# def listen_and_recognize(lang="ne-NP"):
#     mic = sr.Microphone(device_index=1)
#     with mic as source:
#         recognizer.adjust_for_ambient_noise(source)
#         audio = recognizer.listen(source)
#     try:
#         output=recognizer.recognize_google(audio, language=lang)
#     except sr.RequestError:
#         output="API unavailable"
#     except sr.UnknownValueError:
#         output="recognition err"
#     return output

# def listen_and_save(num, lang="ne-NP"):
#     mic = sr.Microphone(device_index=1)
#     with mic as source:
#         audio = recognizer.listen(source)
#         filename="recorded/rec_"+str(num)+".wav"
#         with open(filename, "wb") as f:
#             f.write(audio.get_wav_data())

# #listen and recognize here
# for i in range(0, 4):
#     temp="questions/text_"+str(i)+".mp3"
#     playsound(temp)
#     print(listen_and_recognize(i))
    
# #listen and save here
# for i in range(0, 4):
#     temp="questions/text_"+str(i)+".mp3"
#     playsound(temp)
#     print(listen_and_save(i))